In [12]:
import pandas as pd
import warnings
import altair as alt
from urllib import request
import json

In [13]:
# fetch & enable a Spanish timeFormat locale.
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-ES.json') as f:
  es_time_format = json.load(f)
alt.renderers.set_embed_options(timeFormatLocale=es_time_format)

#warnings.filterwarnings('ignore')

df = pd.read_csv ('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv')

In [14]:
country = 'Mexico'
region = 'Colima'
sub_df = df[(df['country_region']== country) & (df['sub_region_1']==region)]
sub_df.loc[:,'date'] = pd.to_datetime(sub_df.loc[:,'date'])

sub_df

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
1628378,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-02-15,6.0,5.0,4.0,-4.0,2.0,-1.0
1628379,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-02-16,7.0,5.0,1.0,-2.0,-1.0,0.0
1628380,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-02-17,1.0,5.0,-1.0,-9.0,11.0,-2.0
1628381,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-02-18,-2.0,0.0,1.0,-7.0,12.0,-2.0
1628382,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-02-19,-2.0,0.0,-4.0,-8.0,11.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628648,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-11-11,-23.0,-8.0,-30.0,-29.0,-22.0,9.0
1628649,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-11-12,-23.0,-8.0,-31.0,-29.0,-22.0,10.0
1628650,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-11-13,-22.0,-3.0,-29.0,-31.0,-19.0,8.0
1628651,MX,Mexico,Colima,NaN,NaN,MX-COL,NaN,2020-11-14,-19.0,2.0,-21.0,-16.0,-5.0,4.0


In [15]:
# Change and filter date here
sub_df = sub_df[(sub_df['date'] > '2020-03-15') & (sub_df['date'] < '2020-11-15')]
sub_df = sub_df.sort_values('date', ascending=True)

In [16]:
%run urban_theme.py

#Lugares de ocio
retail_recretation = alt.Chart(sub_df).mark_line(size=1).encode(
    alt.X('date:T', title = " "),
    alt.Y('retail_and_recreation_percent_change_from_baseline:Q', title = " "),
).properties(
    title = "Lugares de ocio",
    width = 450,
    height = 250
)

#Mercados y farmacias
grocery_pharmacy = alt.Chart(sub_df).mark_line(size=1).encode(
    alt.X('date:T', title = " "),
    alt.Y('grocery_and_pharmacy_percent_change_from_baseline:Q', title = " "),
).properties(
    title = "Mercados y farmacias",
    width = 450,
    height = 250
)

#Parques y playas 
parks = alt.Chart(sub_df).mark_line(size=1).encode(
    alt.X('date:T', title = " "),
    alt.Y('parks_percent_change_from_baseline:Q', title = " ")
).properties(
    title = "Parques y playas",
    width = 450,
    height = 250
)

#Transporte público
transit = alt.Chart(sub_df).mark_line(size=1).encode(
    alt.X('date:T', title = " "),
    alt.Y('transit_stations_percent_change_from_baseline:Q', title = " ")
).properties(
    title = "Transporte público",
    width = 450,
    height = 250
)

#Lugares de trabajo
workplaces = alt.Chart(sub_df).mark_line(size=1).encode(
    alt.X('date:T', title = " "),
    alt.Y('workplaces_percent_change_from_baseline:Q', title = " ")
).properties(
    title = "Lugares de trabajo",
    width = 450,
    height = 250
)

#Residenciales
residential = alt.Chart(sub_df).mark_line(size=1).encode(
    alt.X('date:T', title = " "),
    alt.Y('residential_percent_change_from_baseline:Q', title = " ")
).properties(
    title = "Residenciales",
    width = 450,
    height = 250
)


In [17]:
par1 = retail_recretation | grocery_pharmacy | parks
par2 = transit | workplaces | residential

mobility = par1 & par2

In [18]:
# Title of the concatened grpah
mobility_1 = mobility.properties(
    title={
        "text":["Movilidad en Colima"],
        "subtitle": ["Datos del 15 de Marzo al 15 de noviembre de 2020.", " "],
    }
)

In [19]:
#Add footer
alt.concat(mobility_1).properties(
    title=alt.TitleParams(
        ['Fuente: Elaboración propia con datos del COVID-19 Community Mobility Reports de Google.', 'Jay Ballesteros (@jballesterosc_)'],
        baseline='bottom',
        orient='bottom',
        anchor='end',
        fontWeight='normal',
        fontSize=12
    )
)

alt.ConcatChart(...)